In [ ]:
import numpy as np
from skimage import io, transform
from pathlib import Path
from skimage.filters import threshold_local
import imageio

# ---------------------------------------------------
# CONFIG
# ---------------------------------------------------
jpg_folder = Path(r"D:\DMMMSU\Thesis2025\another 3D\testset\test\tomo_00e047")     # <-- CHANGE THIS
png_folder = Path("converted_png1")            # output folder
png_folder.mkdir(exist_ok=True)

resize_shape = (300, 300)

mask_method = "superclean"   # adaptive | percentile | manual | superclean
manual_threshold_value = 0.3
percentile_value = 20
# ---------------------------------------------------

In [2]:
# PART 1 — Convert JPG → PNG
# ---------------------------------------------------
jpg_files = sorted(jpg_folder.glob("*.jpg"))
print("Found JPG files:", len(jpg_files))

if len(jpg_files) == 0:
    raise ValueError("❌ No .jpg files found. Check folder path!")

for f in jpg_files:
    img = io.imread(f)
    png_path = png_folder / (f.stem + ".png")
    imageio.imwrite(png_path, img)
    print("Converted:", f.name, "→", png_path.name)

print("\n✔ JPG → PNG conversion complete.\n")


Found JPG files: 300
Converted: slice_0000.jpg → slice_0000.png
Converted: slice_0001.jpg → slice_0001.png
Converted: slice_0002.jpg → slice_0002.png
Converted: slice_0003.jpg → slice_0003.png
Converted: slice_0004.jpg → slice_0004.png
Converted: slice_0005.jpg → slice_0005.png
Converted: slice_0006.jpg → slice_0006.png
Converted: slice_0007.jpg → slice_0007.png
Converted: slice_0008.jpg → slice_0008.png
Converted: slice_0009.jpg → slice_0009.png
Converted: slice_0010.jpg → slice_0010.png
Converted: slice_0011.jpg → slice_0011.png
Converted: slice_0012.jpg → slice_0012.png
Converted: slice_0013.jpg → slice_0013.png
Converted: slice_0014.jpg → slice_0014.png
Converted: slice_0015.jpg → slice_0015.png
Converted: slice_0016.jpg → slice_0016.png
Converted: slice_0017.jpg → slice_0017.png
Converted: slice_0018.jpg → slice_0018.png
Converted: slice_0019.jpg → slice_0019.png
Converted: slice_0020.jpg → slice_0020.png
Converted: slice_0021.jpg → slice_0021.png
Converted: slice_0022.jpg → slice

In [3]:
# PART 2 — Load PNGs and resize to 300×300
# ---------------------------------------------------
png_files = sorted(png_folder.glob("*.png"))
imgs = []

for f in png_files:
    img = io.imread(f)

    # If RGB or RGBA → convert to grayscale
    if img.ndim == 3:
        img = img[..., 0]

    img = img.astype(np.float32) / 255.0
    img_r = transform.resize(img, resize_shape, anti_aliasing=True)
    imgs.append(img_r)

volume = np.stack(imgs)
print("Loaded resized volume:", volume.shape)


Loaded resized volume: (300, 300, 300)


In [7]:
# PART 3 — Apply Masking
# ---------------------------------------------------
from skimage import filters, morphology
vol = volume.copy()
mask_method = "superclean"

if mask_method == "adaptive":
    masked = np.zeros_like(vol)
    for i in range(vol.shape[0]):
        t = threshold_local(vol[i], block_size=35, offset=0.01)
        masked[i] = vol[i] * (vol[i] > t)
    print("Applied adaptive masking.")

elif mask_method == "percentile":
    base_thresh = np.percentile(vol, percentile_value)
    thresh = base_thresh * 0.63
    masked = vol * (vol < thresh)
    print("Applied percentile masking:", thresh)

elif mask_method == "manual":
    masked = vol * (vol < manual_threshold_value)
    print("Applied manual masking:", manual_threshold_value)

elif mask_method == "three":
    blur = filters.gaussian(vol, sigma=1.5)
    t = filters.threshold_li(blur)
    binary = blur < t
    cleaned = morphology.remove_small_objects(binary, min_size=150)
    cleaned = morphology.binary_closing(cleaned, morphology.disk(3))
    masked = vol * cleaned
    print("Applied three.", cleaned)

elif mask_method == "superclean":
    masked = np.zeros_like(vol)

    for i in range(vol.shape[0]):
        slice_ = vol[i]

        # Gaussian blur per slice
        blur = filters.gaussian(slice_, sigma=1.5)

        # Li threshold per slice
        t = filters.threshold_li(blur)
        binary = blur > t

        # Invert mask: True = background, False = foreground
        binary = ~binary

        # Remove small objects (remove background noise)
        cleaned = morphology.remove_small_objects(binary, min_size=150)

        # Close holes in background
        cleaned = morphology.binary_closing(cleaned, morphology.disk(3))

        # Invert again to get foreground mask
        foreground_mask = ~cleaned

        masked[i] = slice_ * foreground_mask

    print("Applied FIXED SUPER-CLEAN mask: background removed, bacteria kept.")


else:
    raise ValueError("Invalid mask_method selected.")


Applied FIXED SUPER-CLEAN mask: background removed, bacteria kept.


In [8]:
# PART 4 — Save Final Volume
# ---------------------------------------------------
np.save("volume_raw1.npy", vol.astype(np.float32))
np.save("volume_masked1.npy", masked.astype(np.float32))

print("\n✔ Saved volume_raw.npy and volume_masked.npy")


✔ Saved volume_raw.npy and volume_masked.npy
